In [0]:
!pip install hyperopt

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import  color, exposure

import tensorflow as tf

from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import  to_categorical

import os, datetime
from sklearn.metrics import  accuracy_score

from hyperopt import  STATUS_OK, fmin, hp, Trials, tpe
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
cd  "/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [7]:
train = pd.read_pickle('data/train.p')
X_train, y_train = train['features'], train['labels']

test = pd.read_pickle('data/test.p')
X_test, y_test = test['features'], test['labels']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((34799, 32, 32, 3), (34799,), (4410, 32, 32, 3), (4410,))

In [8]:
if y_train.ndim == 1: y_train  = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]
input_shape, num_classes

((32, 32, 3), 43)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding='same'),    
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
    MaxPool2D(),    
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
    MaxPool2D(),    
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
  ])

def predict(model, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  return scoring(y_test_norm, y_pred)

def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  return model

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 7s 24ms/step - loss: 2.7027 - accuracy: 0.2683 - val_loss: 1.0444 - val_accuracy: 0.6509
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.8796 - accuracy: 0.7198 - val_loss: 0.2672 - val_accuracy: 0.9156
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.4332 - accuracy: 0.8627 - val_loss: 0.1252 - val_accuracy: 0.9652
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2944 - accuracy: 0.9099 - val_loss: 0.0803 - val_accuracy: 0.9785
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2173 - accuracy: 0.9341 - val_loss: 0.0489 - val_accuracy: 0.9853


0.9666666666666667

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding='same'),
    
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
    MaxPool2D(),    
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation="relu"),
    MaxPool2D(),    
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dens_block_one']),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dens_block_two']),
    Dense(num_classes, activation='softmax')
  ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
      )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy: {}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [36]:
space = {
    'batch_size': hp.quniform('batch_size',50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',.2, .5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',.2, .5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',.2, .5),
    'dropout_dens_block_one': hp.uniform('dropout_dens_block_one',.3, .7),
    'dropout_dens_block_two': hp.uniform('dropout_dens_block_two',.3, .7)
}

best = fmin(func_obj, space, tpe.suggest, 20, Trials())

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.4787583882626281, 'dropout_cnn_block_three': 0.25862405268433813, 'dropout_cnn_block_two': 0.25105903619533476, 'dropout_dens_block_one': 0.4482447491620546, 'dropout_dens_block_two': 0.5345086792302748}
accuracy: 0.9666666388511658
{'batch_size': 60.0, 'dropout_cnn_block_one': 0.2919421867396337, 'dropout_cnn_block_three': 0.39764996978270684, 'dropout_cnn_block_two': 0.4884942241354754, 'dropout_dens_block_one': 0.6218715550517919, 'dropout_dens_block_two': 0.6001435774695829}
accuracy: 0.9383220076560974
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.4550369429397391, 'dropout_cnn_block_three': 0.3240822821339045, 'dropout_cnn_block_two': 0.20051402150818154, 'dropout_dens_block_one': 0.62710209686346, 'dropout_dens_block_two': 0.5082758681495602}
accuracy: 0.9507936239242554
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.4828003490416043, 'dropout_cnn_block_three': 0.263020974630871, 'dropout_cnn_block_two': 0.40930792271757843, 